# 1) **  Installation and Setup **

- Clone the git repository

- Install pest

- Unit->Example.Test --> inside of this code , php test unit la iruka code ah  Pest form ku maatha porom

# 2) ** LifeCycle Hooks **

 [Overview of Pest Hooks]

- beforeEach(): Runs before each individual test.
- afterEach(): Runs after each individual test.
- beforeAll(): Runs once before all tests in a test suite.
- afterAll(): Runs once after all tests in a test suite.

* In software testing, we also need to prepare everything before we run the tests and clean up everything after the tests are done.

* Testing hooks are like the instructions that say, "Hey, do this before you start testing" and "Do this after you're done testing."

* This helps save time and makes testing easier. You don’t have to worry about always setting things up or cleaning things up yourself because the computer does it for you automatically.
(Just like baking cookies, you don't want to have to prepare everything manually every time, and you don’t want to have to clean up your mess by yourself. 
Testing hooks make everything automatic — you only need to focus on your actual tests, and the hooks handle the boring stuff for you.)


ithula Mukkiyama entha Hook ah epo use pannanum nu therinjukanum..
beforeAll and afterAll Hooks, dont have access to any laravel features..

> then we are using beforeEach hooks and afterEach hooks

# 3) ** Expect Api **

we are going to test the contacts

-> Feature-> Contacts -> StoreTest(Create this test)

intha test la Expect api use pannirukom
    
```php
    <?php
    ...
    expect($contact->first_name)->toBeString()->not->toBeEmpty();  #there is no notToBeEmpty , that's why we are using like this.. before the 
    expect($contact->last_name)->toBeString()->not->toBeEmpty();
    expect($contact->email)->toBeString()->toContain('@');
    expect($contact->phone)->toBeString()->toStartWith('+');
    expect($contact->city)->toBe('TesterField');
    expect($contact->region)->toBe('DerbyShire');
    expect($contact->country)->toBeIn(['us', 'ca']);

# 4)  ** Faker **

[Refactoring Code]

we are going to see about faker() method

```php
<?php

** Example **
 $this->post('/contacts', [
        'first_name' =>  $faker->firstName(),
        'last_name' => $faker->lastName(),
        'email' =>$faker->email(),
        'phone' =>$faker->e164PhoneNumber(),
        'address' => '1 Test Street',
        'city' => 'TesterField',
        'region' => 'DerbyShire',
        'country' =>$faker->randomElement(['us', 'ca']),
        'postal_code' =>$faker->postcode(),
  ....
  //use this
use Faker\Factory as Faker;

$faker = Faker::create();   #(this should be in inside of the test)

- Why we using faker() function instead of fake() global function , becoz we can use that faker() function in php unit test too.. 
this faker function customizly created for testing , thats why we are using faker function

antha phone number innum konjam strong aakalam.. 

expect($contact->phone)->toBePhoneNumber();

intha mari kudutha error varum.. becoz apdi  oru api method ey illa.. 

so namma atha create pannuvom

- Pest.php  -> epdi function ku oru place irunthucho athey mari expectations yeluthavum thaniya place irukum

```php
    <?php

    // Expectations
    expect()->extend('toBePhoneNumber', function(){
        expect($this->value)->toBeString()->toStartWith('+');

        if (strlen($this->value) < 6){
            throw new ExpectationFailedException('Phone numbers must be at least 6 characters');
        }

        if (! is_numeric(Str::of($this->value)->after('+')->remove([' ',-''])->__toString())) {
            throw new ExpectationFailedException('phone numbers must be numeric');
        }

    });
//import these things -> 
use PHPUnit\Framework\ExpectationFailedException;
use Illuminate\Support\Str;

```php
<?php

//if you encounter error

// use this ->
    $number = preg_replace('/[^\d]/', '', substr($this->value, 1));
    if (!is_numeric($number)) {
        throw new ExpectationFailedException('Phone numbers must be numeric');
    }


//instead of this ->
 	  if (! is_numeric(Str::of($this->value)->after('+')->remove([' ',-''])->__toString())) {
        throw new ExpectationFailedException('phone numbers must be numeric');
    }
    ...


** THIS IS COSTUM EXPECTATION **
```php
<?php 

// instead of write like this !
  expect($contact->first_name)->toBeString()->not->toBeEmpty();
    expect($contact->last_name)->toBeString()->not->toBeEmpty();
    expect($contact->email)->toBeString()->toContain('@');
    expect($contact->phone)->toBePhoneNumber();             # i write this in Pest.php
    expect($contact->city)->toBe('TesterField');
    expect($contact->region)->toBe('DerbyShire');
    expect($contact->country)->toBeIn(['us', 'ca']);

// we can simplify.. 


    expect($contact)
        ->first_name->toBeString()->not->toBeEmpty()
        ->last_name->toBeString()->not->toBeEmpty()
        ->email->toBeString()->toContain('@')
        ->phone->toBePhoneNumber()
        ->city->toBe('TesterField')
        ->region->toBe('DerbyShire')
        ->country->toBeIn(['us', 'ca']);

//for more about learning expectations .. https://pestphp.com/docs/expectations


# 5) ** DataSets **

Imagine a situation .. one of our costumers solranga , nalla use aagitu iruntha email address thidernu work aagama poiruchu namma website la apdinu.. 

we need to fix this.. 

we are going to now test the email address

ella time um namma email address ah change pannitey iruka koodathu code la,
(duplictaions la remove pannanum)

then how can we test different emails??

ipo namaku 
```php
<?php
'email' => $email ;
?>
```
then intha email variable(value) ah parameter ah test functionla pass panni use panra mari iruntha (dynamic ah) nalla irukum la .. 
athuku than 

- Datasets

    In Pest, datasets send data to the test function through arguments

end of the test using "->with()" chain method 
we can pass within array

intha array la enaku so many emails ah pass pannanum nu nenaikiren so,

```php
<?php
->with([
        [fake()->email()],
        ['luke.downing@downing.tech']
]);

Run this test .. it works 

terminal la note panna we have two tests, then description dynamic ah change aagirukum 

   PASS  Tests\Feature\Contacts\StoreTest

  ✓ it can store a contact with ('fjohnston@gmail.com')                                                                                                        0.63s  
  ✓ it can store a contact with ('luke.downing@downing.tech')      
intha maari.. 

ipo dataset enna panlam nu pathom... (email address)

        then ipo namaku multiple pieces of data va pass pannanum naa enna panrathu..?

yeah namme multiple dataset parameters ah pass panlam.. 

test function kulla , Parameter la $email , pakkathula $firstName yum serthu pass panna porom. 
then ,
```php
<?php
...
->with([
        [fake()->email(), 'Steve'],
        ['luke.downing@downing.tech', 'Rogers'],
        ['stranger@rj.com', 'Rajesh']
]);

see the output

intha mari multiple datasets ah pass panlam.. 
then mail nd name test pass aagum pothu kaaata venam , athuku bathila vera namaku require yethuvo (like in sentence) antha mari katanum nu nenacha
atha oru key value pair la kudukalam intha mari !

```php
<?php
...
->with([
'email for steve' => ['herosare@alive.com', 'firstName'=>'John'],
'email for Rogers' => ['luke.downing@downing.tech', 'firstName'=> 'Steve'],
'email for Rajesh' =>['stranger@rj.com', 'firstName'=> 'Rajesh'],

so namku ipo intha mari ! output kedaikum

   PASS  Tests\Feature\Contacts\StoreTest
   
  ✓ it can store a contact with dataset "email for steve"                                                                                 0.64s  
  ✓ it can store a contact with dataset "email for Rogers"                                                                                0.03s  
  ✓ it can store a contact with dataset "email for Rajesh" 

then innonu ennanaa nammalala dataset ah reuse pannika mudiyum.. multiple test ku.. ovoru test laiyum copy paste panrthuku bathila..

that is 
# "Shared data Set"

>php artisan pest:dataset Emails 

(if this command is not working use these

>composer remove pestphp/pest --dev
>composer remove pestphp/pest-plugin-laravel --dev
>composer require pestphp/pest --dev
>composer require pestphp/pest-plugin-laravel --dev
)
tests->Datasets->Emails.php

```php
<?php

dataset('valid emails', function () {
    return [
        'herosare@alive.com',
        'luke.downing@downing.tech',
        'stranger@rj.com'
    ];
});
<?
```
this valid emails key we are using in inside of the with (->with('valid emails');) 
Now run the test , it works .. 
(Pending Multiple keys passing and testing)

# 6) ** 

one more dataset is here .. combined data set

pingcrm open panna anga profile pakkathula oru photo irukum.. atha click panna..
url la w=60 h=60 antha mari irukum.. 
namma itha edit pic size change aagum (dynamic ah)
itha thaan test panna porom.. 

--ShowTest.php--    (ithuku munnadi fixtures ndra folder la 3 images add pannikalam, atha than test panna porom)

```php
<?php

use Illuminate\Support\Facades\Storage;

it('can show supported image formats and options', function ($path, $options) {
    Storage::fake()->put($path, file_get_contents(__DIR__ . "/../../fixtures/{$path}"));

    $response = $this->get(route(s'image', ['path' => $path, ...$options]));
    $response->assertOk();

    expect($response->streamedContent())->not->toBeEmpty()->toBeString();
})->with([
    ['path'=>'example.png',['w'=>40, 'h'=>40, 'fit'=>'crop']],
    ['path'=>'example.jng',['w'=>40, 'h'=>40, 'fit'=>'crop']],
    ['path'=>'example.webp',['w'=>40, 'h'=>40, 'fit'=>'crop']],
    ['path'=>'example.png',['w'=>50, 'h'=>50, 'fit'=>'crop']],
    ['path'=>'example.jpg',['w'=>50, 'h'=>50, 'fit'=>'crop']],
    ['path'=>'example.webp',['w'=>50, 'h'=>50, 'fit'=>'crop']],
    
]);

- Innum intha mari yeluthitey pogalam (dataset ku).. Becoz 3 images ku nd width and height ku yegapatta possibilities iruku.. yellamey namma yeluthitey iruka mudiyathu
then unreadable aavum maridum intha dataset..

ithuku enna sollution naa athu than 
# Combined DataSets
    Combined DataSets is Cartisian Product of Given Data.. 
        Cartisian Product naa enna naa.. Ipo oru rendu box iruku oru box la pant1, pant2, pant3 iruku nu assume pannikalam.. innoru box la shirt1, shirt2 , shirt3 irku nu assume pannikalam.. 
        ipo namma rendu box la irunthu ovoru time um ovoru pant and shirt yedukurom.. appo yethana possibile combinations create aagum athu than .. 

Ok namma kitta irukatha 2 seperate dataset ah pirikka porom.. then atha combine panna porom.. 
oru dataset- example.png/jng/webp  
other one - w, h , fit

```php
<?php
})->with([
    'example.png',
    'example.jpg',
    'example.webp'
    ])
->with([
    [['w'=>40, 'h'=>40, 'fit'=>'crop']],
    [['w'=>50, 'h'=>50, 'fit'=>'crop']],
    [['w'=>10, 'h'=>20, 'fit'=>'crop']]

]);
```
Run the test and see the output .. rendu different ana datasets um sernthu Cartesian Product ah aagiruchu.. 
Two or more datasets ah use pannum pothu , neraiya possibilities kedaikum apdi nu namaku therinchunaa.. Combined DataSets ah use pannikalam .. its very useful



# 7) **

email address ku terrible ah oru rule yeluthirukom la atha poi konjam fix panlaam.. 

App->Rules->IsValidEmailAddress.php

create LaracastsTest.php (inga than atha test panna porom)

first-u basic ah ithu work aaguthaanu check pannuvom...
```php
<?php

use App\Rules\IsValidEmailAddress;

it('can validate an email', function() {
    $rule = new IsValidEmailAddress();

    expect($rule->passes('email', 'me@you.com'))->toBe(True);

    });
```
it works .. 

then test ah laam group panni kooda test panlaam.. 

> ->group('laracasts') 

intha maari entha entha test laam run pannanum nu nenaikuromo athuku laam same group name('laracasts) kuduthuttu run panlam

> php artisan test --group=laracasts



then athey mari intha laracasts ndra name la iruka groups laam mattum run panna venam , atha skip pannitu mathathellam run pannanaum apdi nu nenacha..

> php artisan test --exclude=laracasts


then ipo oru test la iruka ella test kum orey group la vachu test pannanum nu nenacha ..

uses ()->group('laracasts); (write this in top of the test)

>php artisan test --group=laracasts


Then Regular Expressions ah improve panna porom for the costum rule, so that is actually works with more email addresses as we discussed in previous episodes

we are going to write another test 
```php
<?php
it('has better regex support and can catch more email addresses');
```
then itha run panni pakalam .. (without a content)

Note panni paatha laravel intha test ah fail pannirukathu.. athuku bathila incomplete nu than sollirukum.. 
So testers laam enna pannuvanganaa, firstu test ku vendiya features laam yeluthi vachutu aproma than test yelutha start pannuvanga.. 

OK now back to IsValidEmailAddress.php
anga pathom naa..  

```php
<?php
if (! is_string($value)) {
            throw new InvalidArgumentException('The value must be a string!');
        }
```
pass panra value string ah illana, it throws InvalidArgumentException 

itha epdi pest la test panrathunu papom.. 
```php
<?php
it('throws an exception if the value is not string', function(){
    $rule = new IsValidEmailAddress();

    $rule->passes('email', 1);

    });
```
ithu error varum coz 1 is not a string

then ithulaiye invalid argument ah iruntha pass pannu nu solrom naa... 
```php
 $this->expectException(InvalidArgumentException::class); #write this after- $rule = new IsValidEmailAddress();
```
 Now Try to run the test it passes.. Becoz Argument valid ah illanaa test ah pass panna sollirukom

# * Throw *
 In Pest We can use a throw (chain method)

 ```php
 ->throws(InvalidArgumentException::class)
 ```
 Once we write this we can remove this ( $this->expectException(InvalidArgumentException::class);) from top.. 

 Innoru Best Vishayam ennanaa.. namma throw pannum pothu messageum serthu send panalam.. 

  ```php
 ->throws(InvalidArgumentException::class, 'The value must be a int!')
 ```
 ipdi kudukum pothu error serthu message um show aagum... int ah string nu maathuna sari aagidum.. 
 
 exception ah intha mari end of the test la than kudukanum.. Top of the test la ($this->expectException(InvalidArgumentException::class);) intha mari kudukka koodathu!

 One More Thing.. 
    Test ah Skip panra mari irunthaa epdi irukum.. skipping is useful for lots of cases
    (Sometimes namme test ah finish panniruka matom, or fix panna yethavathu irukum .. antha mari timela laam skip panni than aganum.. production ku push pannum pothu..)

# skip    

 - Skip Method receives a boolean condition, which is either a straight-up boolean or closure. then Messsage venum na kooda add Pannikalam (optional)

 Why dont we skip with no condition.. 

```php
    ->skip('We no longer want to test the exception);     #add this before throws
```    

What if we wanr some conditional check in orderto determine whether we skip or not.. 

```php
->skip(getenv('SKIP_TESTS')) ?? false, 'We no longer want to test the exception')
```



# 8) ** Coverage and Parallel**

- Imagine a big puzzle:

    - Coverage is like checking how many pieces of the puzzle you’ve completed. It shows you which parts of your code are tested and helps spot missing pieces.

    - Parallel is like working on the puzzle with friends—everyone tackles different sections at once, making it faster to finish!

    - So, coverage helps you check all the parts, and parallel makes testing quicker.


Normally coverage coming from , Xml or JSON file.. But in Pest , Beautiful coverage driver for the console

itha use panrathuku namaku 'Xdebug' or 'PCOV' venum.. 

-------------------------------------------------------------------------

# * Installation *

1. Check Your PHP Version

> php -v

2. Download Xdebug

> https://xdebug.org/download

Choose the Windows binary matching your PHP version

     Save the .dll file to your PHP extensions folder. 

            C:\xampp\php\ext

3. Edit php.ini to Enable Xdebug   

Open your php.ini file:

For XAMPP, it's usually located at C:\xampp\php\php.ini

    Add the following lines at the end of the php.ini file:

            [Xdebug]
            zend_extension="C:\xampp\php\ext\php_xdebug-3.1.0-8.1-ts-x86_64.dll"  ; Use the correct path and filename
            xdebug.mode=coverage
            xdebug.start_with_request=yes
            xdebug.client_host=127.0.0.1
            xdebug.client_port=9003
            xdebug.log_level=0

4. Restart Xampp

-------------------------------------------------------------------------

> $ php artisan test --coverage

You can see the coverage test 

Ipo Neraiya test irukathunaala, test ah run pannum pothu, slow ah execute aaguthu.. itha epdi fast panrathu?

thats why we using "Parallel"

Oru Oru test ah execute panrathuku bathila parallel ah panna porom.. this increases the speeed

    athuku mothalla parallel plugin ah download pannanum..

        > composer require --dev pestphp/pest-plugin-parallel --dev --with-all-dependencies

    Now try to execute the test 
        > php artisan test --parallel

        > php artisan test --parallel --coverage

executing speeed was incresed.. 










